<a href="https://colab.research.google.com/github/Cherryberry112/CSE475_Machine_Learning-/blob/main/Lab/lab01_DecisionTree_RandomForest_Mushroom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

# 2. VISUAL THEME (Personalized)

In [ ]:
sns.set_theme(style="whitegrid")
sns.set_palette("pink")
plt.rcParams["figure.figsize"] = (8, 5)